# Ушной метод триангуляция многоугольников

В теме посвященной [трингуляции монотонного многоугольника(будет ссылка на конспект Лёни)]() мы доказали существование триангуляции у простого $n$-вершинного многоугольника и собственно разобрали один из способов ее нахождения.
Здесь мы рассмотрим другой довольно интересный, а так же простой в понимании и реализации алгоритм триангулирования многоугольников, называющийся **ушной триангуляцией**.  

## Определение
Три последовательные вершины $v_{i-1}$, $v_{i}$, $v_{i+1}$ многоугольника $P$ называются **ухом**, если треугольник образованный этими вершинами не содержит внутри себя и на границе остальных вершин многоугльника $P$, а так же угол при вершине $v_{0}$, называемой **вершиной уха**, строго меньше $\pi$ (см рис.1). 

![определение уха](./img/def_ear.jpg)

## Теорема (О существовании двух ушей у многоугольника)
> У любого простого $n$-вершинного многоугольника $P$ всегда существуют два непересекающихся между собой уха.

$\triangleright$
<div style="padding-left:40px">
Доказывать будем по индукции по количеству $n$ вершин.
База: $n = 4$.
![База](./img/th1_base.jpg)

Предположим, что для всех многоугольников, количество вершин в которых не больше $n$, теорема верна.
Рассмотрим многоугольник $P$, в котором $n + 1$ вершина.
Далее возможны два случая:
* Произвольная выпуклая вершина $v_{i}$ многоугольника $P$ является ухом.
Отрезав это ухо, мы уменьшим число вершин $P$ на одну.
В результате, получим $n$-вершинный многоугольник $P'$.
По предположению индукции у него существует два непересекающихся уха.
Учитывая, что уши $P'$ являются и ушами $P$, можно утверждать, что для $P$ теорема верна.
![Ухо](./img/th1_case_1.jpg)

* Произвольная выпуклая вершина $v_{i}$ многоугольника $P$ не является ухом.
В таком случае в треугольнике $\Delta v_{i-1}v_{i}v_{i+1}$ лежит хотя бы одна отличная от вершин треугольника вершина из $P$.
Если их несколько, то выберем вершину $q$, которая будет ближе всего к $v_{i}$.
Проведём отрезок $Q$, который разделит $P$ на два многоугольника: $P_{1}$ и $P_{2}$.
В каждом из них будет не более $n$ вершин, следовательно у каждого будет по два непересекающихся уха.
Даже если предположить, что ухо из $P_{1}$ и ухо из $P_{2}$ будут пересекаться по стороне $v_{i}q$, в $P$ всё равно будет не менее двух непересекающихся ушей.
![Не ухо](./img/th1_case_2.jpg)
</div>
$\triangleleft$

## Задача
>Пусть нам дается список вершин простого $n$-вершинного многоугольника, задающий их в порядке обхода против часовой стрелки. Надо затриангулировать этот многоугольник.

### Алгоритм
Идея заключается в последовательном отрезании ушей от многоугольника, пока он не станет треугольником.
Зафиксируем стартовую вершину многоугльника и будем обходить его в поиске ушей, добавляя в стек $S$ вершины, обработанные нами и не являющиеся ушами.
Ломаная, построенная на вершинах из стека, будет отделять отрезанные уши от еще не просмотренных вершин многоугольника. 

Когда при обходе мы находим вершину уха, то отрезаем ее, добавляя диагональ, и переходим к предыдущей вершине из стека.
Если она стала вершиной уха, то отрезаем ее и дальше спускаемся по стеку.
Этот процесс остановится, когда мы либо достигнем стартовой вершины (стек пуст), либо текущая вершина перестанет быть ухом.
Тогда мы переходим к вершине, являющейся концом последней добавленной диагонали, и продолжаем обход многоугольника в поиске ушей.

В итоге, мы придем к тому, что от нерассмотренной части многоугольника останется только одно ребро, а остальная часть будет затриангулирован.  

#### Ушная проверка

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from hidden import *       # Функции, реалзация которых не особо интересна в свете этой темы
from answers import *      # Ответы
from visual_utils import * # Функции визуализации

In [60]:
# Примеры многоугольников
examples = {
        'some': [(4, 0), (2, 1), (3, 6), (0, 5), (-1, 6), (-3, 4), (-1, 2), (-2, 0),
              (-4, 1), (-5, -2), (-2, -4), (0, -3), (2, -4), (1, -1), (3, -2)]
        ,
        'square': [(0,0), (1,0), (1,1), (0,1)]
        ,
        'arrow': [(0,0), (2,3), (4,0), (2,6)]
        ,
        'star': [(0,0), (7,1), (7,3), (6,5), (5,6), (3,7), (1,7)]
        ,
        'substar': [(0,0), (1,6), (7,1), (7,3), (6,5), (5,6), (3,7), (1,7)]
        ,
        'hook': [(0,3), (2,4), (3,3), (3,2), (2,1), (2,2), (2,3), (1,2), (1,1), (3,0), (4,4), (2,5), (1,4), (0,5)]
        }

# Вершины многоугольника в порядке обхода против часовой стрелки
source = examples['some']
# Преобразовываем пары в наш класс
source = list(map(lambda i: Vertex(i[0], i[1]), source))
# Многоугольник хранится в DCEL
D = build_dcel(source)
# Добавленные диагонали будем хранить в отдельном списке для наглядной визуализации
D1 = []
# Стек вершин
S = []   

In [61]:
def break_into_pieces_triang(folder='.ear_clipping'):
    dump = create_dump_func(folder, visual_dump_ear_clipping_triangulation, D, D1, S)
    
    def last_h():
        return S[len(S) - 1]
    
    def is_ear(hedge, probable_ear, pred):
        def out_of_triangle(v0,v1,v2,v):
            return turn(v0, v1, v)<= -1 or turn(v1, v2, v) <= -1 or turn(v2, v0, v) <= -1 
        
        while pred(hedge):
            if out_of_triangle(*probable_ear, hedge.origin):
                hedge = hedge.next
            else:
                return False 
        return True 
    
    start = D[0]
    dump(start.origin)
    S.append(start)    

    while len(D1) != len(D) - 2:
        cur = last_h().next
        while True:
            # At the start point 
            if len(S) == 0:
                dump(cur.origin)
                if len(D1) == len(D) - 2:
                    break
                S.append(cur)
                cur = cur.next
                
            v0 = last_h().origin
            v1 = cur.origin
            v2 = cur.next.origin
            
            if turn(v0, v1, v2) >= 1 and is_ear(cur.next.next, probable_ear=[v0, v1, v2],
                                                pred=lambda hedge: hedge != start):
                dump(cur.origin)

                prev = S.pop()                
                d = add_diagonal_with_next(prev, cur.next)
                D1.append(d)
                dump(cur.origin)

                cur = d
#                 dump(cur.origin)
            else:
                dump(cur.origin)
                S.append(cur)
                cur = cur.next
                break
                
    # Remove last edge from of last triangle
    S.clear()
    dump(None)

In [62]:
break_into_pieces_triang()

In [63]:
from IPython.display import display
display(SlideShower('.ear_clipping'))